In [1]:
%matplotlib inline

In [2]:
import numpy as np
import tensorflow as tf

from batcher import Batcher
from metrics import acc012

In [3]:
np.random.seed(1337)

In [5]:
X_train = np.load('../../data/bin/train/deal.npy')
y_train = np.load('../../data/bin/train/tricks_spades.npy')

X_val = np.load('../../data/bin/val/deal.npy')
y_val = np.load('../../data/bin/val/tricks_spades.npy')
X_val = X_val.reshape((X_val.shape[0], -1))
y_val = y_val.reshape((y_val.shape[0], 1))

n_examples = X_train.shape[0]

X_train = X_train.reshape((n_examples, -1))
y_train = y_train.reshape((n_examples, 1))

n_dim = X_train.shape[1]
assert(n_dim == 208)

batch_size = 64
n_iterations = 10000
display_step = 1000

learning_rate = 0.001

In [6]:
X = tf.placeholder(tf.float32, [None, n_dim])
Y = tf.placeholder(tf.float32, [None, 1])

In [7]:
W = tf.Variable(np.zeros((n_dim, 1)), dtype=tf.float32, name='weights')

In [8]:
pred = tf.matmul(X, W)

In [9]:
rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(pred, Y)))

In [10]:
cost = tf.reduce_mean(tf.squared_difference(pred, Y))

In [11]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [12]:
init = tf.global_variables_initializer()

In [13]:
batch = Batcher(n_examples, batch_size)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = batch.next_batch([X_train, y_train])
        
        sess.run(optimizer, feed_dict={X:x_batch, Y:y_batch})
        
        if iteration % display_step == 0:
            c = sess.run(cost, feed_dict={X: X_train, Y: y_train})
            rms_error = sess.run(rmse, feed_dict={X: X_train, Y: y_train})
            pred_train = sess.run(pred, feed_dict={X: X_train, Y: y_train})
            pred_val = sess.run(pred, feed_dict={X: X_val, Y: y_val})
            print('it={} cost={} rmse={}'.format(iteration, c, rms_error))
            print(acc012(y_train, pred_train))
            print(acc012(y_val, pred_val))
            
    predictions = sess.run(pred, feed_dict={X: X_train, Y: y_train})
        